In [18]:
 # Initial imports
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, mean_squared_error, r2_score
%matplotlib inline

In [19]:
# Loading data
ob_df = pd.read_csv("Nutrition__Physical_Activity__and_Obesity_-_Behavioral_Risk_Factor_Surveillance_System.csv")
ob_df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,Data_Value_Type,...,GeoLocation,ClassID,TopicID,QuestionID,DataValueTypeID,LocationID,StratificationCategory1,Stratification1,StratificationCategoryId1,StratificationID1
0,2020,2020,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who engage in no leisure-tim...,NaN,Value,...,NaN,PA,PA1,Q047,VALUE,59,Race/Ethnicity,Hispanic,RACE,RACEHIS
1,2014,2014,GU,Guam,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,"(13.444304, 144.793731)",OWS,OWS1,Q036,VALUE,66,Education,High school graduate,EDU,EDUHSGRAD
2,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q036,VALUE,59,Income,"$50,000 - $74,999",INC,INC5075
3,2013,2013,US,National,Behavioral Risk Factor Surveillance System,Obesity / Weight Status,Obesity / Weight Status,Percent of adults aged 18 years and older who ...,NaN,Value,...,NaN,OWS,OWS1,Q037,VALUE,59,Income,Data not reported,INC,INCNR
4,2015,2015,US,National,Behavioral Risk Factor Surveillance System,Physical Activity,Physical Activity - Behavior,Percent of adults who achieve at least 300 min...,NaN,Value,...,NaN,PA,PA1,Q045,VALUE,59,Income,"Less than $15,000",INC,INCLESS15


In [20]:
ob_clean_df = ob_df[['YearStart', 'LocationAbbr',       
       'Data_Value',
       #'Low_Confidence_Limit', 'High_Confidence_Limit ',
       #'Sample_Size', 
       #'TopicID',
       'QuestionID',
       'StratificationID1']].copy()

In [21]:
ob_clean_df = ob_clean_df[(ob_clean_df['StratificationID1'] != 'OVERALL') & (ob_clean_df['LocationAbbr'] != 'US') & (ob_clean_df['Data_Value'].notna())].copy()
ob_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75110 entries, 1 to 88627
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   YearStart          75110 non-null  int64  
 1   LocationAbbr       75110 non-null  object 
 2   Data_Value         75110 non-null  float64
 3   QuestionID         75110 non-null  object 
 4   StratificationID1  75110 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.4+ MB


In [22]:
ob_clean_df = ob_clean_df.pivot(index=["YearStart", "LocationAbbr", "StratificationID1"], columns=["QuestionID"], values="Data_Value")
ob_clean_df.head()

QuestionID                                Q018  Q019  Q036  Q037  Q043  Q044  \
YearStart LocationAbbr StratificationID1                                       
2011      AK           AGEYR1824           NaN   NaN  19.8  32.0  62.8  34.9   
                       AGEYR2534           NaN   NaN  23.5  38.7  57.6  27.0   
                       AGEYR3544           NaN   NaN  29.5  38.9  55.3  22.2   
                       AGEYR4554           NaN   NaN  29.2  43.3  56.2  25.2   
                       AGEYR5564           NaN   NaN  33.4  38.9  58.2  20.1   

QuestionID                                Q045  Q046  Q047  
YearStart LocationAbbr StratificationID1                    
2011      AK           AGEYR1824          32.4  51.9  16.1  
                       AGEYR2534          35.7  36.4  18.1  
                       AGEYR3544          32.3  30.8  21.1  
                       AGEYR4554          38.8  32.4  24.7  
                       AGEYR5564          43.3  26.6  26.0

In [23]:
ob_clean_df = ob_clean_df.reset_index()
ob_clean_df

QuestionID,YearStart,LocationAbbr,StratificationID1,Q018,Q019,Q036,Q037,Q043,Q044,Q045,Q046,Q047
0,2011,AK,AGEYR1824,NaN,NaN,19.8,32.0,62.8,34.9,32.4,51.9,16.1
1,2011,AK,AGEYR2534,NaN,NaN,23.5,38.7,57.6,27.0,35.7,36.4,18.1
2,2011,AK,AGEYR3544,NaN,NaN,29.5,38.9,55.3,22.2,32.3,30.8,21.1
3,2011,AK,AGEYR4554,NaN,NaN,29.2,43.3,56.2,25.2,38.8,32.4,24.7
4,2011,AK,AGEYR5564,NaN,NaN,33.4,38.9,58.2,20.1,43.3,26.6,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14127,2021,WY,MALE,45.4,21.2,32.4,41.2,NaN,NaN,NaN,NaN,20.5
14128,2021,WY,RACE2PLUS,53.9,39.8,43.5,51.4,NaN,NaN,NaN,NaN,23.1
14129,2021,WY,RACEHIS,49.6,21.0,39.0,33.9,NaN,NaN,NaN,NaN,30.3
14130,2021,WY,RACENAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.6


In [24]:
#Q018, #Q019 = fruit and vegetable questions
ob_clean_df = ob_clean_df.loc[ob_clean_df["YearStart"].isin([2011,2013,2015,2017,2019])]
ob_clean_df = ob_clean_df.drop(columns=["Q018", "Q019"])
ob_clean_df = ob_clean_df.dropna()
ob_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6289 entries, 0 to 11523
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   YearStart          6289 non-null   int64  
 1   LocationAbbr       6289 non-null   object 
 2   StratificationID1  6289 non-null   object 
 3   Q036               6289 non-null   float64
 4   Q037               6289 non-null   float64
 5   Q043               6289 non-null   float64
 6   Q044               6289 non-null   float64
 7   Q045               6289 non-null   float64
 8   Q046               6289 non-null   float64
 9   Q047               6289 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 540.5+ KB


In [25]:
ob_dummies_df = pd.get_dummies(ob_clean_df)
ob_dummies_df.head()

,YearStart,Q036,Q037,Q043,Q044,Q045,Q046,Q047,LocationAbbr_AK,LocationAbbr_AL,...,StratificationID1_INCNR,StratificationID1_MALE,StratificationID1_RACE2PLUS,StratificationID1_RACEASN,StratificationID1_RACEBLK,StratificationID1_RACEHIS,StratificationID1_RACEHPI,StratificationID1_RACENAA,StratificationID1_RACEOTH,StratificationID1_RACEWHT
0,2011,19.8,32.0,62.8,34.9,32.4,51.9,16.1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2011,23.5,38.7,57.6,27.0,35.7,36.4,18.1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2011,29.5,38.9,55.3,22.2,32.3,30.8,21.1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2011,29.2,43.3,56.2,25.2,38.8,32.4,24.7,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2011,33.4,38.9,58.2,20.1,43.3,26.6,26.0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
ob_dummies_df["Years_since_start"] = 2023 - ob_dummies_df['YearStart']
del ob_dummies_df['YearStart']
ob_dummies_df.head()

,Q036,Q037,Q043,Q044,Q045,Q046,Q047,LocationAbbr_AK,LocationAbbr_AL,LocationAbbr_AR,...,StratificationID1_MALE,StratificationID1_RACE2PLUS,StratificationID1_RACEASN,StratificationID1_RACEBLK,StratificationID1_RACEHIS,StratificationID1_RACEHPI,StratificationID1_RACENAA,StratificationID1_RACEOTH,StratificationID1_RACEWHT,Years_since_start
0,19.8,32.0,62.8,34.9,32.4,51.9,16.1,1,0,0,...,0,0,0,0,0,0,0,0,0,12
1,23.5,38.7,57.6,27.0,35.7,36.4,18.1,1,0,0,...,0,0,0,0,0,0,0,0,0,12
2,29.5,38.9,55.3,22.2,32.3,30.8,21.1,1,0,0,...,0,0,0,0,0,0,0,0,0,12
3,29.2,43.3,56.2,25.2,38.8,32.4,24.7,1,0,0,...,0,0,0,0,0,0,0,0,0,12
4,33.4,38.9,58.2,20.1,43.3,26.6,26.0,1,0,0,...,0,0,0,0,0,0,0,0,0,12


In [27]:
print(ob_dummies_df.shape)

ob_dummies_df = ob_dummies_df.dropna()

print(ob_dummies_df.shape)

(6289, 88)
(6289, 88)


In [28]:
#Q036 = Percent of adults aged 18 years and older who have obesity
# Select features and target variable
X = ob_dummies_df.drop("Q036", axis=1)
y = ob_dummies_df["Q036"]



In [47]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [48]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [49]:
# Create the Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100)


In [50]:
# Train the model
model.fit(X_train_scaled, y_train)


RandomForestRegressor()

In [51]:
# Make predictions
y_pred = model.predict(X_test_scaled)


In [52]:
# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Target variable standard deviation is {y_test.std()}")
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R^2 Score:", r2)

Target variable standard deviation is 7.601110822685729
Mean Squared Error: 13.974148244038156
Root Mean Squared Error: 3.738201204327846
R^2 Score: 0.7579435940815924
